In [15]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error
import joblib

In [16]:
# Load the data
df = pd.read_parquet('data/clean/df.parquet').drop(columns=['24h_later_forecast'])
df.head(3)

,forecast_dt,24h_later_load
datetime,,
2014-12-14 00:00:00,2014-12-15 00:00:00,6131
2014-12-14 01:00:00,2014-12-15 01:00:00,5842
2014-12-14 02:00:00,2014-12-15 02:00:00,5715


In [18]:
# Enrich the df with the datetime attributes
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['hour'] = df.index.hour
df['weekday'] = df.index.weekday
df.head(3)

,forecast_dt,24h_later_load,year,month,day,hour,weekday
datetime,,,,,,,
2014-12-14 00:00:00,2014-12-15 00:00:00,6131,2014,12,14,0,6
2014-12-14 01:00:00,2014-12-15 01:00:00,5842,2014,12,14,1,6
2014-12-14 02:00:00,2014-12-15 02:00:00,5715,2014,12,14,2,6


In [19]:
def compute_timedelta_ago_load(df, timedelta):
    """For each timestamps in the index, compute the load timedelta ago 

    Assume that each row's index is the current timestamp.
    That is, when we say "timedelta ago from now", we mean "timedelta ago from this timestamp".

    df (pd.DataFrame): Dataframe containing the `24h_later_load`, whose index refers to now when saying "24h later".
    timedelta (pd.Timedelta): Time delta of interest, i.e. how long ago do we want the load ?
    """
    
    assert '24h_later_load' in df.columns
    assert isinstance(df.index, pd.DatetimeIndex)

    ts_to_24h_later_load = df['24h_later_load'].to_dict()
    return df.index.to_series().apply(lambda x: ts_to_24h_later_load.get(x - pd.Timedelta(24, 'h') - timedelta))

In [20]:
# Enrich each row with previous loads: 1h-ago, 2h-ago, 3h-ago, 24h-ago, 7days-ago
df['1h_ago_load'] = compute_timedelta_ago_load(df, timedelta=pd.Timedelta(1, 'h'))
df['2h_ago_load'] = compute_timedelta_ago_load(df, timedelta=pd.Timedelta(2, 'h'))
df['3h_ago_load'] = compute_timedelta_ago_load(df, timedelta=pd.Timedelta(3, 'h'))
df['24h_ago_load'] = compute_timedelta_ago_load(df, timedelta=pd.Timedelta(24, 'h'))
df['7d_ago_load'] = compute_timedelta_ago_load(df, timedelta=pd.Timedelta(7, 'd'))
df.head(3)

,forecast_dt,24h_later_load,year,month,day,hour,weekday,1h_ago_load,2h_ago_load,3h_ago_load,24h_ago_load,7d_ago_load
datetime,,,,,,,,,,,,
2014-12-14 00:00:00,2014-12-15 00:00:00,6131,2014,12,14,0,6,NaN,NaN,NaN,NaN,NaN
2014-12-14 01:00:00,2014-12-15 01:00:00,5842,2014,12,14,1,6,NaN,NaN,NaN,NaN,NaN
2014-12-14 02:00:00,2014-12-15 02:00:00,5715,2014,12,14,2,6,NaN,NaN,NaN,NaN,NaN


In [21]:
def compute_stat(df, timedelta, stat):
    """For each timestamps in the index, compute each statistic in `stats` over the date comprised between now and timedelta ago. 

    Assume that each row's index is the current timestamp.
    That is, when we say "timedelta ago from now", we mean "timedelta ago from this timestamp".

    df (pd.DataFrame): Dataframe containing the `24h_later_load`, whose index refers to now when saying "24h later".
    timedelta (pd.Timedelta): Time delta of interest, i.e. how long ago do we want the statistics calculation to start ?
    stats (list[func]): Functions of the statistic to compute
    """

    assert '24h_later_load' in df.columns
    assert isinstance(df.index, pd.DatetimeIndex)

    def _compute_stat(current_time, timedelta, stat):
        start_time = current_time -  pd.Timedelta(24, 'h') - timedelta
        end_time = current_time - pd.Timedelta(24, 'h') 
        
        relevant_data = df.loc[start_time:end_time, '24h_later_load']
    
        if len(relevant_data) == 0:
            return np.nan
    
        return stat(relevant_data.values)

    return df.index.to_series().apply(lambda x: _compute_stat(x, timedelta, stat))

In [22]:
# Enrich the df with statistics
df['8h_min'] = compute_stat(df, pd.Timedelta(8, 'h'), np.min)
df['8h_max'] = compute_stat(df, pd.Timedelta(8, 'h'), np.max)
df['8h_median'] = compute_stat(df, pd.Timedelta(8, 'h'), np.median)

df['24h_min'] = compute_stat(df, pd.Timedelta(24, 'h'), np.min)
df['24h_max'] = compute_stat(df, pd.Timedelta(24, 'h'), np.max)
df['24h_median'] = compute_stat(df, pd.Timedelta(24, 'h'), np.median)

df['7d_min'] = compute_stat(df, pd.Timedelta(7, 'd'), np.min)
df['7d_max'] = compute_stat(df, pd.Timedelta(7, 'd'), np.max)
df['7d_median'] = compute_stat(df, pd.Timedelta(7, 'd'), np.median)
df.head(3)

,forecast_dt,24h_later_load,year,month,day,hour,weekday,1h_ago_load,2h_ago_load,3h_ago_load,...,7d_ago_load,8h_min,8h_max,8h_median,24h_min,24h_max,24h_median,7d_min,7d_max,7d_median
datetime,,,,,,,,,,,,,,,,,,,,,
2014-12-14 00:00:00,2014-12-15 00:00:00,6131,2014,12,14,0,6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-14 01:00:00,2014-12-15 01:00:00,5842,2014,12,14,1,6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-14 02:00:00,2014-12-15 02:00:00,5715,2014,12,14,2,6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df = df.dropna()
df.head(3)

,forecast_dt,24h_later_load,year,month,day,hour,weekday,1h_ago_load,2h_ago_load,3h_ago_load,...,7d_ago_load,8h_min,8h_max,8h_median,24h_min,24h_max,24h_median,7d_min,7d_max,7d_median
datetime,,,,,,,,,,,,,,,,,,,,,
2015-01-08 00:00:00,2015-01-09 00:00:00,7299,2015,1,8,0,3,8000.0,8212.0,8568.0,...,7673.0,7605.0,9361.0,8695.0,7347.0,9447.0,8695.0,6380.0,9447.0,7496.0
2015-01-08 01:00:00,2015-01-09 01:00:00,6999,2015,1,8,1,3,7605.0,8000.0,8212.0,...,7633.0,7499.0,9361.0,8568.0,7347.0,9447.0,8695.0,6380.0,9447.0,7496.0
2015-01-08 02:00:00,2015-01-09 02:00:00,6864,2015,1,8,2,3,7499.0,7605.0,8000.0,...,7654.0,7423.0,9361.0,8212.0,7347.0,9447.0,8695.0,6380.0,9447.0,7466.0


In [24]:
# Build Xy
Xy = df[[
    'year', 'month', 'day', 'hour', 'weekday', # daily attributes
    '1h_ago_load', '2h_ago_load', '3h_ago_load', '24h_ago_load', '7d_ago_load', # past loads
    '8h_min', '8h_max', '8h_median', '24h_min', '24h_max', '24h_median', '7d_min', '7d_max', '7d_median', # statistics
    '24h_later_load' # y
]]
Xy.head(3)

,year,month,day,hour,weekday,1h_ago_load,2h_ago_load,3h_ago_load,24h_ago_load,7d_ago_load,8h_min,8h_max,8h_median,24h_min,24h_max,24h_median,7d_min,7d_max,7d_median,24h_later_load
datetime,,,,,,,,,,,,,,,,,,,,
2015-01-08 00:00:00,2015,1,8,0,3,8000.0,8212.0,8568.0,7722.0,7673.0,7605.0,9361.0,8695.0,7347.0,9447.0,8695.0,6380.0,9447.0,7496.0,7299
2015-01-08 01:00:00,2015,1,8,1,3,7605.0,8000.0,8212.0,7528.0,7633.0,7499.0,9361.0,8568.0,7347.0,9447.0,8695.0,6380.0,9447.0,7496.0,6999
2015-01-08 02:00:00,2015,1,8,2,3,7499.0,7605.0,8000.0,7510.0,7654.0,7423.0,9361.0,8212.0,7347.0,9447.0,8695.0,6380.0,9447.0,7466.0,6864


In [26]:
def backtesting(Xy, model, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=1):
    """Backtest the model, starting at starting_ts and add use_every_nth_ts row at each iteration.
    """
    
    cutoff_timestamps = Xy[Xy.index >= starting_ts].index.to_list()
    
    cutoff_ts_to_y = {}
    for cutoff_ts in tqdm(cutoff_timestamps[::use_every_nth_ts]):    
        
        # Split train:val
        Xy_train = Xy[Xy.index < cutoff_ts]
        Xy_val = Xy[Xy.index == cutoff_ts]
        
        # Split X,y
        X_train, y_train = Xy_train.drop(columns=['24h_later_load']), Xy_train['24h_later_load']
        X_val, y_val = Xy_val.drop(columns=['24h_later_load']), Xy_val['24h_later_load']
    
        # Train model
        model.fit(X_train, y_train)
    
        # Compute prediction in 24h
        yhat_val = model.predict(X_val) 
    
        cutoff_ts_to_y[cutoff_ts] = (yhat_val[0], y_val.iloc[0])
        
    results_df = pd.DataFrame({
        'cutoff_ts': cutoff_ts_to_y.keys(), 
        'predicted_24h_later_load': [e[0] for e in cutoff_ts_to_y.values()], 
        '24h_later_load': [e[1] for e in cutoff_ts_to_y.values()]
    })

    mape = mean_absolute_percentage_error(results_df['24h_later_load'], results_df['predicted_24h_later_load']) * 100

    return results_df, mape

In [27]:
# Train & test model
reg = lgb.LGBMRegressor(n_estimators=100, force_row_wise=True, verbose=0)
results_df, mape = backtesting(Xy, model=reg, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=10)
print(f'Backtested MAPE: {mape:.2f}%')
results_df.head(3)

100%|█████████████████████████████████████████| 121/121 [00:47<00:00,  2.53it/s]

Backtested MAPE: 5.42%


,cutoff_ts,predicted_24h_later_load,24h_later_load
0,2024-08-01 00:00:00,5122.455738,5029
1,2024-08-01 10:00:00,5585.366736,4821
2,2024-08-01 20:00:00,4947.960893,4987


In [11]:
# Train & test model
reg = lgb.LGBMRegressor(n_estimators=10_000, force_row_wise=True, verbose=0)
results_df = backtesting(Xy, model=reg, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=10)
results_df.head(3)

100%|█████████████████████████████████████████| 121/121 [29:34<00:00, 14.66s/it]


,cutoff_ts,predicted_24h_later_load,24h_later_load
0,2024-08-01 00:00:00,5270.831067,5029
1,2024-08-01 10:00:00,5432.906974,4821
2,2024-08-01 20:00:00,4826.473411,4987


In [14]:
# Dump
joblib.dump(reg, 'model.joblib')

['model.joblib']